In [15]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
from openai import OpenAI
import os
import json
import random
import time

In [30]:
# load data
data = pd.read_excel("data/intermediate/openai_results.xlsx")

In [67]:
# Defining the categories and their corresponding overcategories
# Daten fuer die Tabelle vorbereiten
categoories_dict = {
    "Ueberkategorie": ["Arbeitsumfeld und -ausstattung"] * 10 + ["Arbeitsorganisation und -bedingungen"] * 10 + ["Kommunikation und soziale Interaktion"] * 10 + ["Fuehrung und Unternehmenskultur"] * 10 + ["Mitarbeiterwohlbefinden und -services"] * 10,
    "Kategorie": [
        "Buerogestaltung und -ausstattung", "Technische Ausstattung und Arbeitsmittel", "Ergonomie am Arbeitsplatz", "Sauberkeit und Ordnung", "Raeumlichkeiten und Standort",
        "Laermpegel und Schallschutz", "Klimatisierung und Temperatur", "Großraumbueros und private Arbeitsbereiche", "Parkplatz, Kantine, Teekueche", "Sicherheits- und Gesundheitsmanagement",
        "Arbeitszeitgestaltung und Gleitzeit", "Homeoffice und flexible Arbeitsorte", "Urlaubsplanung und Freizeitausgleich", "Work-Life-Balance", "ueberstundenmanagement",
        "Schichtarbeit und Pausenregelungen", "Aufgabenbereich und Projekte", "Teamarbeit und Gruppenprojekte", "Selbststaendiges Arbeiten", "Herausforderungen und Abwechslungsreichtum",
        "Kommunikationsstil und -wege", "Umgangston und Arbeitsatmosphaere", "Zusammenarbeit im Team", "Feedbackkultur und Lob", "Meetings und Besprechungen",
        "Offenheit und Transparenz", "Konfliktmanagement", "Interkulturelle Zusammenarbeit", "Soziale Events und Teambuilding", "Netzwerk und Kontaktmoeglichkeiten",
        "Verhalten und Stil der Fuehrungskraefte", "Mitarbeiterfuehrung und -unterstuetzung", "Vertrauen und Respekt", "Wertschaetzung und Anerkennung", "Mitarbeiterentwicklung und -foerderung",
        "Innovationskultur und Kreativitaetsfoerderung", "Unternehmenswerte und Philosophie", "Entscheidungsfindung und Mitbestimmung", "Gerechtigkeit und Fairness", "Unternehmenskommunikation",
        "Gesundheitsfoerderung und Praevention", "Psychologische Unterstuetzung", "Arbeitszufriedenheit und Motivation", "Persoenliche Entwicklung und Weiterbildung", "Familiaere Atmosphaere und sozialer Zusammenhalt",
        "Betriebliche Sozialleistungen und Benefits", "Integration neuer Mitarbeiter", "Mitarbeiterfeedback und -befragungen", "Erreichbarkeit und Hilfsbereitschaft", "Mitarbeiterveranstaltungen und Freizeitangebote"
    ]
}

# Creating the dataframe
categoories = pd.DataFrame(categoories_dict)

categoories = categoories.reset_index(drop=True)
categoories['category_id'] = categoories.index + 1
categoories.to_excel("data/processed/categories.xlsx", index=False)

In [68]:
categoories

,Ueberkategorie,Kategorie,category_id
0,Arbeitsumfeld und -ausstattung,Buerogestaltung und -ausstattung,1
1,Arbeitsumfeld und -ausstattung,Technische Ausstattung und Arbeitsmittel,2
2,Arbeitsumfeld und -ausstattung,Ergonomie am Arbeitsplatz,3
3,Arbeitsumfeld und -ausstattung,Sauberkeit und Ordnung,4
4,Arbeitsumfeld und -ausstattung,Raeumlichkeiten und Standort,5
5,Arbeitsumfeld und -ausstattung,Laermpegel und Schallschutz,6
6,Arbeitsumfeld und -ausstattung,Klimatisierung und Temperatur,7
7,Arbeitsumfeld und -ausstattung,Großraumbueros und private Arbeitsbereiche,8
8,Arbeitsumfeld und -ausstattung,"Parkplatz, Kantine, Teekueche",9
9,Arbeitsorganisation und -bedingungen,Arbeitszeitgestaltung und Gleitzeit,10


In [61]:

# set function output format
class FindCategoryAiResponse(BaseModel):
    category: str
    category_id: int
    probability: float

    
# set openai api key
client = OpenAI(
  api_key="",  # this is also the default, it can be omitted
)

In [62]:
# Define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        num_retries = 0
        delay = initial_delay

        while True:
            try:
                return func(*args, **kwargs)

            except e:
                num_retries += 1

                if num_retries > max_retries:
                    raise Exception(f"Maximum number of retries ({max_retries}) exceeded.")

                delay *= exponential_base * (1 + jitter * random.random())

                time.sleep(delay)

            except Exception as e:
                raise e

    return wrapper

In [69]:

@retry_with_exponential_backoff
def call_open_ai(client: OpenAI, statement_df: pd.Series, categoories: pd.DataFrame) -> dict:
    
    row = categoories.rename(columns={'Kategorie': 'category'})
    category_json = row[['category_id', 'category']].to_json(orient='records', force_ascii=False)
    
    prompt = f""" Kategorien: {category_json} \n
    Aussage: {statement_df['value']}\n 
    key_aspect: {statement_df['key_aspect']}\n
    """
    
    # print(prompt)
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4-1106-preview",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        functions=[
            {
            "name": "find_most_fitting_category",
            "description": """Aus den vorgegebenen Kategorien, finde die am besten passende Kategorie für die Aussage auf einer Jobportal Seite.""",
            "parameters": FindCategoryAiResponse.model_json_schema()
            }
        ],
        function_call={"name": "find_most_fitting_category"},
        temperature = 0.2,
        timeout=30,
    )

    completion_value = completion.choices[0].message.function_call.arguments
    completion_dict = json.loads(completion_value)
    completion_dict = completion_dict
    
    return completion_dict
    


In [70]:
results = []
data_df = data.copy()
data_df['category_id '] = None
data_df['category'] = ""
data_df['category_probability'] = None
category_ids = categoories['category_id'].unique()

for index, row in data_df.iterrows():
    
    # ignore missing values
    if row['value'] is np.nan:
        continue
    
    # call open ai
    result = call_open_ai(client, row, categoories)
    results.append(result)
    
    if result['category_id'] not in category_ids:
        print('Category Not found')
        continue
    
    if categoories[categoories['category_id'] == result['category_id']]['Kategorie'].values[0] != result['category']:
        print(f"{categoories[categoories['category_id'] == result['category_id']]['Kategorie'].values[0]} - {result['category']}")
    
    print(f"{index}: {row['key_aspect']} - {row['short_sumary']}")
    print(result['category'])
    print("----------")
    print()
    row['category_id '] = result['category_id']
    row['category'] = categoories[categoories['category_id'] == result['category_id']]['Kategorie'].values[0]
    row['category_probability'] = result['probability']
    data_df.loc[index] = row
    
    if index % 20 == 0:
        # result_df = pd.DataFrame(results)
        data_df.to_excel("data/processed/coded_data_3.xlsx")

data_df.to_excel("data/processed/coded_data_3.xlsx")
data_df    

Familiaere Atmosphaere und sozialer Zusammenhalt - Atmosphäre
0: Atmosphäre - Herzlich, freundlich, hilfsbereit und positiv
Atmosphäre
----------

Category Not found
Teamarbeit und Gruppenprojekte - Teamwork
2: Teamwork - Sehr hilfsbereit und auch neben dem Job trinkt man mal was zusammen.
Teamwork
----------

3: Vielseitigkeit - Arbeit mit Menschen
Herausforderungen und Abwechslungsreichtum
----------

Category Not found
Category Not found
6: Arbeitszeit - Balance zwischen Arbeit und Freizeit
Arbeitszeitgestaltung und Gleitzeit
----------

Category Not found
Category Not found
Zusammenarbeit im Team - Teamzusammenhalt
9: Teamzusammenhalt - Familiäres Arbeitsumfeld mit Spaß und Scherzen
Teamzusammenhalt
----------

Category Not found
Category Not found


UnboundLocalError: cannot access local variable 'e' where it is not associated with a value

In [15]:
result_df = pd.DataFrame(results)
result_df.to_excel("data/openai_results.xlsx")

In [7]:
data

,Unnamed: 0,topic,short_sumary,key_aspect,key_word,positve,rating,assagekraft,sarcasm_rating,value,document,column_name
0,0,Arbeitsatmosphäre,"Herzlich, freundlich, hilfsbereit und positiv",Atmosphäre,lebendig,True,9,8,False,"Herzlich, freundlich, hilfsbereit und voralle...",1,working_atmosphere_comment
1,1,Mentoring,"Super Mentorin, die persönliche Weiterentwickl...",Mentoring,Unterstützung,True,9,8,False,"Ich habe einer super Mentorin , die mir die Mö...",1,leadership_behavior_comment
2,2,Arbeitsatmosphäre,Sehr hilfsbereit und auch neben dem Job trinkt...,Teamwork,gemeinschaftlich,True,8,7,False,Sehr hilfsbereit und auch neben dem Job trinkt...,1,team_spirit_comment
3,3,Beratung,Arbeit mit Menschen,Vielseitigkeit,Spaß,True,8,7,False,Bei der Beratung steht der Mensch im Mittelpun...,1,interesting_tasks_comment
4,4,Freundlichkeit,Die Mitarbeiter waren sehr freundlich und hilf...,Mitarbeiter,freundlich,True,8,9,False,"Freundlich, konstruktiv und höflich.",1,communication_comment
...,...,...,...,...,...,...,...,...,...,...,...,...
1164,1164,Kollegen und Zusammenhalt,Klasse Zusammenhalt,Zusammenarbeit,Kollegen,True,9,8,False,Die Kollegen und der Zusammenhalt ist klasse!,200,team_spirit_comment
1165,1165,Arbeitsinhalt,Die Arbeit macht Spaß und ist vielseitig,workload,hoch,True,8,7,False,Die Arbeit macht Spaß und ist vielseitig...es ...,200,interesting_tasks_comment
1166,1166,Transparenz,Manchmal fehlt die Transparenz,Informationsweitergabe,spät,False,7,5,False,"Allgemein gut, nur manchmal fehlt die Transpar...",200,communication_comment
1167,1167,Arbeitsumgebung,"Systeme hängen ab und an, sonst alles top! Küc...",Systemstabilität,ab und an,False,8,7,False,"Systeme hängen ab und an, sonst alles top! Küc...",200,working_conditions_comment


In [13]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [14]:
str(set(data[data['positve']]['topic'].to_list() + data[data['positve']]['key_aspect'].to_list()))

"{'Vertrauen und Wertschätzung', 'Unterstützung und Informationsfluss', 'Stellenbeschreibung', 'Vertrauen und Wohlbefinden', 'Gestaltung', 'Vorgesetzte', 'Schulungen und Weiterbildung', 'Tagesablauf', 'Erweiterungsmöglichkeiten', 'Verhalten der Chefs', 'Gleitzeit und Urlaubsanspruch', 'Umgangston und Miteinander', 'Weitergabe von Informationen', 'Zusammenarbeit mit verschiedenen Kulturen', 'Unterstützung außerhalb des Arbeitsalltags', 'Feedbackkultur', 'Urlaub und Arzttermine', 'Ausstattung und Angebote', 'Bezahlung', 'Gestaltungsmöglichkeiten und Weiterentwicklung', 'Montage', 'Rückendeckung von Beraterin und Leiterin', 'Arbeitsplatz', 'Unterhaltung', 'Interessante und vielfältige Aufgaben', 'Usability', 'Gleitzeit', 'Überstundenabbau und -ausgleich', 'Verbesserungen', 'Verständnis für Probleme', 'Schichtplanung', 'Büros/Räumlichkeiten', 'Förderung', 'Zusammenhalt im Team', 'Beziehung zwischen Arbeitern und Vorgesetzten', 'Arbeitsplatzgestaltung', 'Jugendschutzgesetz', 'Freundlichkeit